# Progetto data Analytics

## Installing libraries

In [0]:
#for install libraries
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
libxml2 set to manually installed.
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
     |████████████████████████████████| 71kB 2.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built cairocffi
     |████████████████████████████████| 378kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eeba

#### Execute this only if you are Alex Caselli on Colab

In [43]:
from google.colab import drive
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
drive.mount('/content/gdrive')
baseURL = '/content/gdrive/My Drive/Data Analytics/';

ModuleNotFoundError: No module named 'google'

## Import Libraries and set local path

In [13]:
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
import csv
from tqdm import tqdm_notebook as tqdm

baseURL = '../'

# Creation of the Total Graph and the Alliance Graph
You don't have to run this code if you already have all the graph files in your dataset


## Create compact attack graph

In [45]:
# Creation of 30 plain graphs
nGraph = 30
compactAttackGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

attackCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    attackGraph = nx.MultiDiGraph()
    compactAttackGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(attackGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(attackGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                attackCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            attackCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactAttackGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(attackGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAttackGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAttackGraph, baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553

FROM VERTICES NUMBER IN DICTIONARY:  938
START GRAPH ALL VERTICES:  2807
VERTICES WITH OUTDEGREE 0: 1869

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2807
Number of edges: 7476
Average in degree:   2.6633
Average out degree:   2.6633

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  32997
NUMBER OF STEP (END GRAPH EDGES NUMBER):  7476
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995

FROM VERTICES NUMBER IN DICTIONARY:  927
START GRAPH ALL VERTICES:  2802
VERTICES WITH OUTDEGREE 0: 1875

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2802
Number of edges: 7177
Average in degree:   2.5614
Average out degree:   2.5614

SUM OF

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1992
Number of edges: 19969
Average in degree:  10.0246
Average out degree:  10.0246

FROM VERTICES NUMBER IN DICTIONARY:  764
START GRAPH ALL VERTICES:  1992
VERTICES WITH OUTDEGREE 0: 1228

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1992
Number of edges: 4587
Average in degree:   2.3027
Average out degree:   2.3027

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  19969
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4587
NUMBER OF ATTACK EDGES MET:  19969
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1990
Number of edges: 22134
Average in degree:  11.1226
Average out degree:  11.1226

FROM VERTICES NUMBER IN DICTIONARY:  742
START GRAPH ALL VERTICES:  1990
VERTICES WITH OUTDEGREE 0: 1248

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1990
Number of edges: 4705
Average in degree:   2.3643
Average out degree:   2.3643

SUM 

DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1685
Number of edges: 16513
Average in degree:   9.8000
Average out degree:   9.8000

FROM VERTICES NUMBER IN DICTIONARY:  646
START GRAPH ALL VERTICES:  1685
VERTICES WITH OUTDEGREE 0: 1039

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1685
Number of edges: 4008
Average in degree:   2.3786
Average out degree:   2.3786

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  16513
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4008
NUMBER OF ATTACK EDGES MET:  16513
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1643
Number of edges: 16749
Average in degree:  10.1942
Average out degree:  10.1942

FROM VERTICES NUMBER IN DICTIONARY:  643
START GRAPH ALL VERTICES:  1643
VERTICES WITH OUTDEGREE 0: 1000

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1643
Number of edges: 3727
Average in degree:   2.2684
Average out degree:   2.2684

SUM 

## Create compact trade graph

In [46]:
# Creation of 30 plain graphs
nGraph = 30
compactTradeGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

cont = 0
tradeCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    tradeGraph = nx.MultiDiGraph()
    compactTradeGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(tradeGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(tradeGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                tradeCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            tradeCount += 1

    
    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if fromVertex == "2789":
                print(fromVertex)
                cont += edgesDict[fromVertex][toVertex]
            nStep += 1
            compactTradeGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(tradeGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactTradeGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    nx.write_graphml(compactTradeGraph, baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')
print(cont)
    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284

2789
FROM VERTICES NUMBER IN DICTIONARY:  1209
START GRAPH ALL VERTICES:  1336
VERTICES WITH OUTDEGREE 0: 127

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1336
Number of edges: 4909
Average in degree:   3.6744
Average out degree:   3.6744

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  9390
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4909
NUMBER OF TRADE EDGES MET:  9390
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1314
Number of edges: 8710
Average in degree:   6.6286
Average out degree:   6.6286

2789
FROM VERTICES NUMBER IN DICTIONARY:  1188
START GRAPH ALL VERTICES:  1314
VERTICES WITH OUTDEGREE 0: 126

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1314
Number of edges: 4659
Average in degree:   3.5457
Average out degree:   3.5457

S

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1199
Number of edges: 10175
Average in degree:   8.4862
Average out degree:   8.4862

2789
FROM VERTICES NUMBER IN DICTIONARY:  1097
START GRAPH ALL VERTICES:  1199
VERTICES WITH OUTDEGREE 0: 102

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1199
Number of edges: 4799
Average in degree:   4.0025
Average out degree:   4.0025

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  10175
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4799
NUMBER OF TRADE EDGES MET:  10175
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1146
Number of edges: 9866
Average in degree:   8.6091
Average out degree:   8.6091

2789
FROM VERTICES NUMBER IN DICTIONARY:  1053
START GRAPH ALL VERTICES:  1146
VERTICES WITH OUTDEGREE 0: 93

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1146
Number of edges: 4924
Average in degree:   4.2967
Average out degree:   4.296

DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 991
Number of edges: 8606
Average in degree:   8.6842
Average out degree:   8.6842

FROM VERTICES NUMBER IN DICTIONARY:  896
START GRAPH ALL VERTICES:  991
VERTICES WITH OUTDEGREE 0: 95

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 991
Number of edges: 4125
Average in degree:   4.1625
Average out degree:   4.1625

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8606
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4125
NUMBER OF TRADE EDGES MET:  8606
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 967
Number of edges: 8521
Average in degree:   8.8118
Average out degree:   8.8118

FROM VERTICES NUMBER IN DICTIONARY:  880
START GRAPH ALL VERTICES:  967
VERTICES WITH OUTDEGREE 0: 87

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 967
Number of edges: 4004
Average in degree:   4.1406
Average out degree:   4.1406

SUM OF WEIGHT BEFOR

## Create compact message graph

In [49]:
# Creation of 30 plain graphs
nGraph = 30
compactMessageGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

messageCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    messageGraph = nx.MultiDiGraph()
    compactMessageGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(messageGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(messageGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                messageCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                messageCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            messageCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactMessageGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(messageGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactMessageGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(compactMessageGraph, baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271

FROM VERTICES NUMBER IN DICTIONARY:  1168
START GRAPH ALL VERTICES:  1963
VERTICES WITH OUTDEGREE 0: 795

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1963
Number of edges: 6485
Average in degree:   3.3036
Average out degree:   3.3036

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  16346
NUMBER OF STEP (END GRAPH EDGES NUMBER):  6485
NUMBER OF MESSAGE EDGES MET:  16346
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2100
Number of edges: 15019
Average in degree:   7.1519
Average out degree:   7.1519

FROM VERTICES NUMBER IN DICTIONARY:  1143
START GRAPH ALL VERTICES:  2100
VERTICES WITH OUTDEGREE 0: 957

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2100
Number of edges: 6105
Average in degree:   2.9071
Average out degree:   2.9071

SUM O

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1777
Number of edges: 11805
Average in degree:   6.6432
Average out degree:   6.6432

FROM VERTICES NUMBER IN DICTIONARY:  969
START GRAPH ALL VERTICES:  1777
VERTICES WITH OUTDEGREE 0: 808

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1777
Number of edges: 5302
Average in degree:   2.9837
Average out degree:   2.9837

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  11805
NUMBER OF STEP (END GRAPH EDGES NUMBER):  5302
NUMBER OF MESSAGE EDGES MET:  11805
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1785
Number of edges: 13474
Average in degree:   7.5485
Average out degree:   7.5485

FROM VERTICES NUMBER IN DICTIONARY:  931
START GRAPH ALL VERTICES:  1785
VERTICES WITH OUTDEGREE 0: 854

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1785
Number of edges: 5700
Average in degree:   3.1933
Average out degree:   3.1933

SUM O

FROM VERTICES NUMBER IN DICTIONARY:  775
START GRAPH ALL VERTICES:  1408
VERTICES WITH OUTDEGREE 0: 633

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1408
Number of edges: 3174
Average in degree:   2.2543
Average out degree:   2.2543

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  7587
NUMBER OF STEP (END GRAPH EDGES NUMBER):  3174
NUMBER OF MESSAGE EDGES MET:  7587
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1512
Number of edges: 8468
Average in degree:   5.6005
Average out degree:   5.6005

FROM VERTICES NUMBER IN DICTIONARY:  734
START GRAPH ALL VERTICES:  1512
VERTICES WITH OUTDEGREE 0: 778

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1512
Number of edges: 3492
Average in degree:   2.3095
Average out degree:   2.3095

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8468
NUMBER OF STEP (END GRAPH EDGES NUMBER):  3492
NUMBER OF MESSAGE EDGES MET:  8468
------------

## Create Total and Alliance Graph

In [50]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

attackSelf = 0
messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    attackSelf = 0
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label = "Trade", time = itr[3]['edgetime'])

    #for itr in attackGraph.edges_iter(None, True, True):
    for itr in list(attackGraph.edges(None, True, True)):
        if int(itr[0]) != int(itr[1]):
            realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
        else:
            attackSelf += 1
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(allianceGraph, baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("NO SENSE SELF ATTACK EDGE: ", attackSelf)
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
NO SENSE SELF MESSAGE EDGE:  515
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1833
Number of edges: 14447
Average in degree:   7.8816
Average out degree:   7.8816
NO SENSE SELF MESSAGE EDGE:  402
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1145
Number of edges: 9310
Average in degree:   8.1310
Average out degree:   8.1310
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1996
Number of edges: 23355
Average in degree:  11.7009
Average out degree:  11.7009
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2965
Number of edges: 44983
Average in degree:  15.1713
Average out degree:  15.1713

Graph number:  13
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2066
Number of edges: 20675
Average in degree:  10.0073
Average out degree:  10.0073
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: M

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1732
Number of edges: 16477
Average in degree:   9.5133
Average out degree:   9.5133
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1632
Number of edges: 13178
Average in degree:   8.0748
Average out degree:   8.0748
NO SENSE SELF MESSAGE EDGE:  289
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1013
Number of edges: 8062
Average in degree:   7.9585
Average out degree:   7.9585
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1764
Number of edges: 20951
Average in degree:  11.8770
Average out degree:  11.8770
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2610
Number of edges: 37428
Average in degree:  14.3402
Average out degree:  14.3402

Graph number:  24
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1716
Number of edges: 16467
Average in degree:   9.5962
Average out degree:   9.5962
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: M

## TRANSFORMATION OF THE ALLIANCE MULTIGRAPH TO A PLAIN GRAPH WITH WEIGHTED EDGES

---

Don't run this code if you already have the plain graph in your dataset folder

In [51]:
#Creation of 30 plain graphs
nGraph = 30
realAllianceGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
cont = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    allianceGraph = nx.MultiDiGraph()
    realAllianceGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(allianceGraph.edges(None, True, True)):
        if itr[0] == '7855':
            cont += 1
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
            else:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tradeCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), Weight = sum(edgesDict[fromVertex][toVertex]))
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allianceGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(realAllianceGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(realAllianceGraph, baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)
    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711

FROM VERTICES NUMBER IN DICTIONARY:  1707
START GRAPH ALL VERTICES:  2218
VERTICES WITH OUTDEGREE 0: 511

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2218
Number of edges: 11077
Average in degree:   4.9941
Average out degree:   4.9941

NUMBER OF STEP (END GRAPH EDGES NUMBER):  11077
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2311
Number of edges: 23310
Average in degree:  10.0865
Average out degree:  10.0865

FROM VERTICES NUMBER IN DICTIONARY:  1659
START GRAPH ALL VERTICES:  2311
VERTICES WITH OUTDEGREE 0: 652

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2311
Number of edges: 10448
Average in degree:   4.5210
Average out degree:   4.5210

NUMBER OF STEP (END GRAPH EDGES 

DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1925
Number of edges: 23046
Average in degree:  11.9719
Average out degree:  11.9719

FROM VERTICES NUMBER IN DICTIONARY:  1385
START GRAPH ALL VERTICES:  1925
VERTICES WITH OUTDEGREE 0: 540

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1925
Number of edges: 10303
Average in degree:   5.3522
Average out degree:   5.3522

NUMBER OF STEP (END GRAPH EDGES NUMBER):  10303
NUMBER OF TRADE EDGES MET:  9866
NUMBER OF MESSAGE EDGES MET:  13180
----------------------------------------------------------------------
8
DAY:  17
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1828
Number of edges: 21863
Average in degree:  11.9601
Average out degree:  11.9601

FROM VERTICES NUMBER IN DICTIONARY:  1406
START GRAPH ALL VERTICES:  1828
VERTICES WITH OUTDEGREE 0: 422

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1828
Number of edges: 8992
Average in degree:   4.9190
Average out degree:   4.9190

NUMBER OF STEP (END GRAPH EDGES

## TRANSFORMATION OF THE TOTAL MULTIGRAPH TO A MULTIGRAPH  WITH WEIGHTED EDGES FOR EACH TYPE OF INTERACTION

---

Don't run this code if you already have the plain graph in your dataset folder

In [52]:

nGraph = 30
compactAllGraph = nx.MultiDiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
attackCount = 0
finalCheck = 0
cont = 0

#for each day 
for i in range(0, nGraph):
    
    #create new plaingraphs 
    allGraph = nx.MultiDiGraph()
    compactAllGraph = nx.MultiDiGraph()
    checkGraph = ig.Graph()
    allGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    
    #initialize counters
    tradeCount = 0
    messageCount = 0
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    finalCheck = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allGraph))
    print("")

    #create a tmp dictionary to count the numbers of messages, trades and attacks between each couple of nodes 
    for itr in list(allGraph.edges(None, True, True)):
        # print(itr)
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                    
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]][2] += 1
                    attackCount += 1
            else: 
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                    attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                tradeCount += 1
            elif itr[3]['Label'] == 'Attack':
                edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                attackCount += 1
    if '7855' in edgesDict:
        cont += edgesDict['7855']['3778'][1]
   #For each edges saved in the dictionary add it to the plain graph with the correct label 
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if edgesDict[fromVertex][toVertex][0] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="m", Label = "Message", Weight = edgesDict[fromVertex][toVertex][0])
                nStep += 1
            if edgesDict[fromVertex][toVertex][1] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="t", Label = "Trade", Weight = edgesDict[fromVertex][toVertex][1])
                nStep += 1
            if edgesDict[fromVertex][toVertex][2] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="a", Label = "Attack", Weight = edgesDict[fromVertex][toVertex][2])
                nStep += 1
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

      
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allGraph))

    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAllGraph))
    print("")
    
    #Crazy things of Villa
    for itr in list(compactAllGraph.edges(None, True, True)):
        finalCheck += itr[3]['Weight']
    print("PARANOIC CHECK: ", finalCheck)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAllGraph, baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

FROM VERTICES NUMBER IN DICTIONARY:  1872
START GRAPH ALL VERTICES:  3757
VERTICES WITH OUTDEGREE 0: 1885

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 18792
Average in degree:   5.0019
Average out degree:   5.0019

PARANOIC CHECK:  58218
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  58218
NUMBER OF STEP (END GRAPH EDGES NUMBER):  18792
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 3758
Number of edges: 56092
Average in degree:  14.9260
Average out degree:  14.9260

FROM VERTICES NUMBER IN DICTIONARY:  1836
START GRAPH ALL VERTICES:  3758
VERTICES WITH OUTDEGREE 0: 1922

END GRAPH
Name: 
Type: MultiDiGraph


----------------------------------------------------------------------
6
DAY:  13
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 44180
Average in degree:  15.0426
Average out degree:  15.0426

FROM VERTICES NUMBER IN DICTIONARY:  1599
START GRAPH ALL VERTICES:  2937
VERTICES WITH OUTDEGREE 0: 1338

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 15199
Average in degree:   5.1750
Average out degree:   5.1750

PARANOIC CHECK:  44180
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  44180
NUMBER OF STEP (END GRAPH EDGES NUMBER):  15199
NUMBER OF TRADE EDGES MET:  9298
NUMBER OF MESSAGE EDGES MET:  14207
NUMBER OF ATTACK EDGES MET:  20675
----------------------------------------------------------------------
6
DAY:  14
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2888
Number of edges: 44687
Average in degree:  15.4733
Average out degree:  15.4733

FROM VERTICES NUMBER IN DICTIONARY:  1646
START GRAPH ALL VERTICES: 

----------------------------------------------------------------------
17
DAY:  25
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2525
Number of edges: 31765
Average in degree:  12.5802
Average out degree:  12.5802

FROM VERTICES NUMBER IN DICTIONARY:  1306
START GRAPH ALL VERTICES:  2525
VERTICES WITH OUTDEGREE 0: 1219

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2525
Number of edges: 10788
Average in degree:   4.2725
Average out degree:   4.2725

PARANOIC CHECK:  31765
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  31765
NUMBER OF STEP (END GRAPH EDGES NUMBER):  10788
NUMBER OF TRADE EDGES MET:  8054
NUMBER OF MESSAGE EDGES MET:  7996
NUMBER OF ATTACK EDGES MET:  15715
----------------------------------------------------------------------
18
DAY:  26
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2432
Number of edges: 34297
Average in degree:  14.1024
Average out degree:  14.1024

FROM VERTICES NUMBER IN DICTIONARY:  1261
START GRAPH ALL VERTICES:

## Creation of a dictionaty for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, allGraph, compactallGraph)

In [14]:
#if i can't read from Drive is caused by this
!ls '/content/gdrive/My Drive/Data Analytics/Dataset (1)'

# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()
compactAllGraph = ig.Graph()

for i in tqdm(range(0, nGraph)):
    # print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph, compactAllGraph) 

ls: /content/gdrive/My Drive/Data Analytics/Dataset (1): No such file or directory


Duplicate per Alex

In [18]:
#if i can't read from Drive is caused by this
!ls '/content/gdrive/My Drive/Data Analytics/Dataset (1)'

# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()
compactAllGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Attacks-Network-Graphml/Attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Messages-Network-Graphml/Messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Trades-Network-Graphml/Trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    compactAllGraph = compactAllGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/compactAllGraph/cAllGraph-2009-12-"+str(i+1)+".graphml")
    
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph, compactAllGraph) 

ls: cannot access '/content/gdrive/My Drive/Data Analytics/Dataset (1)': No such file or directory
Day:  1


FileNotFoundError: [Errno 2] No such file or directory: '../Dataset (1)/Graphml-File/Attacks-Network-Graphml/Attacks-timestamped-2009-12-1.graphml'

### This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [15]:
print(len(allNetwork[0][0].vs))
cont = 0
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][4].vs:
        if vertex['id'] == '2789':
            print("{:>20}{:>3}".format("PRESENT IN THE DAY:", str(i + 1)))
            print("{:>20}{:>3}".format("DAY INDEGREE: ", vertex.indegree()))
            print("{:>20}{:>3}".format("DAY OUTDEGREE: ", vertex.outdegree()))          
            for edge in allNetwork[i][4].es:
                if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
                    if edge['Label'] == 'Trade':
                        cont += edge['Weight']
                    print("{:>20}{:>3}".format("OUTDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
                if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
                    print("{:>20}{:>3}".format("INDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))    
            print("") 
print(cont)
    
'''for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
            print(edge)
print("")
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
            print(edge)'''


# 11850

2807
 PRESENT IN THE DAY:  1
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 6.0

 PRESENT IN THE DAY:  2
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 10.0

 PRESENT IN THE DAY:  3
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 9.0

 PRESENT IN THE DAY:  4
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 8.0

 PRESENT IN THE DAY:  5
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 9.0

 PRESENT IN THE DAY:  6
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 8.0

 PRESENT IN THE DAY:  7
      DAY INDEGREE:   0
     DAY OUTDEGREE:   1
   OUTDEGREE LABEL: Trade
          QUANTITY: 7.0

 PRESENT IN THE DAY:  8
      DAY INDEGREE:   2
     DAY OUTDEGREE:   1
    INDEGREE LABEL: Attack
          QUANTITY: 2.0
    INDEGR

'for i in range(0, len(allNetwork)):\n    print("DAY: "+str(i+1))\n    for edge in allNetwork[i][4].es:\n        if allNetwork[i][4].vs[edge.tuple[0]][\'id\'] == \'2789\':\n            print(edge)\nprint("")\nfor i in range(0, len(allNetwork)):\n    print("DAY: "+str(i+1))\n    for edge in allNetwork[i][4].es:\n        if allNetwork[i][4].vs[edge.tuple[1]][\'id\'] == \'2789\':\n            print(edge)'

### Domande da fare
**1)** Il numero massimo e minimo di nodi, dato il paper, è corretto e ci torna; il numero minimo e massimo di archi invece no. Il fatto è molto strano perché noi semplicemente apriamo il file graphml e stampiamo le info associate. Lavoriamo facendo finta di nulla? <br>
1) è stato detto di fregarsene, l'importante è che si usi o il file gml oppure il csv, non bisogna usare entrambi

**2)** Abbiamo osservato come, dato il file di community e la rete totale (attack + trade + message), non tutti i nodi sono presenti in questo file. Indagando su questi abbiamo poi osservato che subiscono (nella maggior parte dei casi) attacchi. Possiamo quindi dedurre che si tratti effettivamente di nodi non affiliati a nessuna alleanza in quanto utenti che hanno smesso di giocare o stanno iniziando a giocare (alcuni non subiscono attacchi ma commerciano e attaccano, scudo prottetivo 30 giorni, punteggio minimo giocare)? <br>
2) è normale in quanto le community segnate sono quelle registrate dagli utenti stessi sul server, quindi è possibile che alcuni nodi non siano affiliati a nessuna alleanza

**3)** Le misure di centralità sono possibili per reti multigrafo? <br>
**4)** In cosa differisce la presentazione dalla demo? <br>
**5)** Le community di dimensione minore di 10 come possiamo tratterle in maniera intelligente? Le associamo tutte a uno stesso identificativo, le eliminiamo, che facciamo?


# START ANALYSIS !!!!

### Users number trend in the 30 days

In [16]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", round(sum(nUsers)/len(nUsers)), 2))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")
    return nUsers

### Edges number trend in the 30 days

In [17]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += edge['Weight']
            elif edge['Label'] == "Trade":
                nTrade[i] += edge['Weight']
            elif edge['Label'] == "Message":
                nMessage[i] += edge['Weight']
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")
    
    return nAttack, nTrade, nMessage

Code for generate a csv with edge and nodes trends in 30days

In [18]:
nAttack, nTrade, nMessage=edgeNumberTrend(allNetwork)
nUsers=userNumberTrend(allNetwork)


#write trend in a CSV
with open(baseURL+'Dash/userAndEdge_trends_30days.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['day', 'users', 'attack', 'trade', 'message'])
    
    for i in range(0, 30):
        filewriter.writerow([i, nUsers[i],nAttack[i], nTrade[i], nMessage[i]])

Number of attacks in the 30 days: [32997.0, 32782.0, 30264.0, 23883.0, 18748.0, 22424.0, 23448.0, 22550.0, 22401.0, 22778.0, 22162.0, 21628.0, 20675.0, 22086.0, 19969.0, 22134.0, 22367.0, 21772.0, 18820.0, 18390.0, 18771.0, 16970.0, 16477.0, 16467.0, 15715.0, 18164.0, 17230.0, 17057.0, 16513.0, 16749.0]

Number of trades in the 30 days: [9390.0, 8710.0, 8667.0, 8545.0, 9499.0, 9799.0, 9638.0, 8806.0, 8698.0, 8921.0, 9008.0, 9310.0, 9298.0, 9834.0, 10175.0, 9866.0, 9884.0, 9008.0, 8624.0, 8314.0, 8443.0, 8981.0, 8062.0, 8659.0, 8054.0, 8436.0, 9062.0, 8665.0, 8606.0, 8521.0]

Number of messages in the 30 days: [15831.0, 14600.0, 14847.0, 12765.0, 14852.0, 14141.0, 13978.0, 14166.0, 15675.0, 14122.0, 12030.0, 14045.0, 14207.0, 12767.0, 11405.0, 13180.0, 11979.0, 12482.0, 14510.0, 10314.0, 11110.0, 13327.0, 12889.0, 11486.0, 7996.0, 7697.0, 9071.0, 8631.0, 7388.0, 8214.0]

   Average number of attacks in 30 days:  21079.7
  Average number of trades a in 30 days:   8982.8
Average number of

### Trickster detection tool 
This tool is designed for identifying dishonest users who manage two accounts and use one for the main account. The assumption is: if i see you two times do more trading than any else action you are a suspected and is necessary to see what are you done in the period (30 days). 

In [21]:
# Main function call all function that compose the tool
def tricksterAnalysis(allNetwork):
    
    # make a list with main suspect for every day, in the end we have a following structure
    #{DAY: {idSuspected: {idBeneficiry: qta}}} 
    # a dictionary where for first external key we have a day, inside 1 lvl key is suspected id, in the last 2 lvl
    # a dictionary of beneficiary key with quantity of trading
    dailySuspicions = dict()
    print("Start trickester Analysis")
    print("Watching for dishonest user in the period...")
    for i in tqdm(range(0, len(allNetwork))):
        # print("DAY: ", str(i+1))
        dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])
    
    # from first skimming compute a guilty index and print the player with more then 0.3 on index
    edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

# create a day dictionary structur (see below)
def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    # get, specific net, all vertex with 0 indegree
    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    # print(zeroIndegreeVertex)
    
    # create a dictionary for keeping suspects with respective beneficiary
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    # for every zero indegree vertex count the number of edge and the number of trade edge
    # if find a trade edge save the beneficiary of this trade and quantity
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += edge['Weight']
                if edge['Label'] == 'Trade':
                #    if totalNet.vs[edge.tuple[0]]['id'] == vertex['id'] == "7855":
                #        print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += edge['Weight']
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = edge['Weight']
                    nTrades += edge['Weight']
        # if the trades edges number is more than 95%, the node is suspected, saved it!
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    # for key in suspecteds:
    #    print(key, suspecteds[key])
    return suspecteds    

# from all day suspects we want a more compact structure for see what happen to our suspects node in the period
# (30 days). The structure is a dictionary: 
# {idSuspected: [nWatchTimes, {idBeneficiary: [nWatchTimes, qnt]}, nWatchSuspectAsBeneficiary, qnt]}
# from this we can delete from suspects list the node with only one appearance
def printRealSuspected(allSuspects, nDays):
    
    print("Compact recovered information...")
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            # increase the number of nWatchTimes about suspects
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            # for all suspect beneficiary increase nWatchTimes about beneficiary and qnt about trade
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    # increase nWatchSuspectAsBeneficiary qnt index
    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    # order from the most observed to the least observed 
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    # print("Complete: ", len(suspectsSummary))
    # for suspect in suspectsSummary:
    #    print(suspect, suspectsSummary[suspect])
    
    # if suspect is observed only one time or is observed as many times as there are beneficiaries; 
    # is no longer a suspect
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
 
# compute a Guiltyindex (based to experimental observations) and print the node with more than .3
def edgesCount(finalSuspects, allNet):
    print("This are the main suspects!")
    edgesInCount = dict()
    edgesOutCount = dict()
    tradeEdgesInCount = dict()
    edgesOutPlayer = dict()
 
    activityDay = list()
    attackDay = list()
    tradeDay = list()
    
    inCount = 0
    outCount = 0
    tradeOutCount = 0
    tradeInCount = 0
    guiltyIndex = 0
    cont = 0
    
    # count number of edge about all typology
    for suspect in finalSuspects:
        
        activityDay.clear()
        attackDay.clear()
        tradeDay.clear()
        
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        tradeEdgesInCount.clear()
        
        for i in range(0, 3):
            edgesInCount['Attack'] = 0
            edgesInCount['Trade'] = 0
            edgesInCount['Message'] = 0
            edgesOutCount['Attack'] = 0
            edgesOutCount['Trade'] = 0
            edgesOutCount['Message'] = 0
        
        inCount = 0
        outCount = 0
        tradeOutCount = 0
        tradeInCount = 0
        guiltyIndex = 0
        
        # append a activity day of suspect
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))
            

            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += edge['Weight']
                    else:
                        edgesOutCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += edge['Weight']
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = edge['Weight']
                        if (day + 1) not in tradeDay:
                            tradeDay.append(day + 1)
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] == 'Trade':
                        if allNet[day][4].vs[edge.tuple[0]]['id'] in tradeEdgesInCount:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] += edge['Weight']
                        else:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] = edge['Weight']
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += edge['Weight']
                    else:
                        edgesInCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Attack':
                        if (day + 1) not in attackDay:
                            attackDay.append(day + 1)
        guiltyIndex = edgesOutCount['Trade']*((1/len(edgesOutPlayer)))*1.3
        # print("Trade out: ", edgesOutCount['Trade'])
        # print("People Served: ", len(edgesOutPlayer))
        # print(guiltyIndex)
        guiltyIndex -= (edgesInCount['Trade']*5)
        # print(guiltyIndex)
        guiltyIndex -= (edgesOutCount['Message'])*1
        # print(guiltyIndex)
        guiltyIndex += (edgesInCount['Message'])*0.3
        # print(guiltyIndex)
        guiltyIndex -= edgesOutCount['Attack']*0.4
        # print(guiltyIndex)
        guiltyIndex -= edgesInCount['Attack']*0.08
        # print(guiltyIndex)
        # print("DENOMINATOR: ", edgesInCount['Attack']+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']+edgesOutCount['Trade'])
        guiltyIndex = guiltyIndex / (edgesInCount['Attack']*0.08+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']*2+edgesOutCount['Trade'])
        if guiltyIndex >= 0.3:
            print("GULTY INDEX: ", round(guiltyIndex, 4))
            print("SUSPECT: ", suspect)
            print("EDGES IN: ", edgesInCount)
            print("EDGES OUT: ", edgesOutCount)
            print("PLAYER SERVED: ", edgesOutPlayer)
            print("ACTIVITY DAYS: ", activityDay)
            print("DAYS OF ATTACKS SUFFERED: ", attackDay)
            print("DAYS OF TRADE DONE: ", tradeDay)
            print("PLAYER TRADE WITH SUSPECT: ", tradeEdgesInCount)
            print(" ")
            cont += 1
    print(cont)

## Communities Dynamics

#### Add communities label to vertex

In [9]:
def addCommunitiesLabel(allCompactNetwork, dayCommunity):
    print("DAY: ", (dayCommunity+1))
    communities = open(baseURL +  "Dataset/Communities/communities-2009-12-"+str(dayCommunity + 1)+".txt", "r")
    communityCont = 1
    nNodes = 0
    noMatchNodes = list()
    present = False
    nNodesPresent = 0
    smallCommunityId = -1
    communityAffiliates = dict()
    affiliatesNodes = list()
    
    nodesGraphNoMatch = list()
    vertexToDelete = list()
    
    for line in communities:
        # print("COMMUNITY NUMBER: ", cont)
        # print(line)
        line = line.split()
        # print(type(line[0]))
        for node in line:
            present = False
            nNodes += 1
            # allCompactNetwork.vs.select('id' = node)['Community'] = communityCont
            for vertex in allCompactNetwork.vs:
                if vertex['id'] == str(node):
                    if len(line) > 10:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = communityCont
                    else:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = -communityCont
                    nNodesPresent += 1
                    present = True
            if not present:
                noMatchNodes.append(node)
       
        communityAffiliates[communityCont] = list(affiliatesNodes)
        communityCont += 1
        affiliatesNodes.clear()
        
    print("NUMBER OF NODES IN DAY TOTAL GRAPH: ", len(allCompactNetwork.vs))
    
    for vertex in allCompactNetwork.vs:
        if vertex['Community'] is None:
            nodesGraphNoMatch.append(vertex['id'])
            vertexToDelete.append(vertex.index)
            # print(vertex.index)
            # print(vertex)
    allCompactNetwork.delete_vertices(vertexToDelete)
        
    print("NUMBER OF NODES IN COMMUNITIES FILE: ", nNodes)
    print("NUMBER OF COMMUNITIES: ", communityCont)
    # print("NODES IN COMMUNITIES DAY FILE NO PRESENT IN DAY GRAPH: ", noMatchNodes)
    print("NUMBER OF NODES IN COMMUNITIES DAY FILE NL PRESENT IN DAY GRAPH: ", len(noMatchNodes))
    # print("NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", nodesGraphNoMatch)
    print("NUMBER OF NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", len(nodesGraphNoMatch))
    print("NUMBER OF NODES IN COMMUNITIES DAY FILE PRESENT IN DAY GRAPH: ", nNodesPresent)
    print("ALL DAY GRAPH AFTER REMOVING THE NO COMMUNITIES NODE: ", allCompactNetwork.summary())
    print("----------------------------------")
    return communityAffiliates

#### Communities War

In [73]:
def find_wars(dayGraph):
  #war = np.array(30,#community, #community)
  #for day in range(0,30)
    #for each attack edge
      #if att['community'] != def['community']
        #war[day, att['community'], def['community']] += 1
    #if war[day, att['community'], def['community']] > #min attack number to call it war
      #print('there was a war between A e B')
      
      
def war_change_communities(war, comm_dict):
    #check if war[day, A, B] --> #members[day + 1, B] < #mebers[day, B]
    
    
def war_change_leader(war, leaders, attackGraph):
    #if war[day, A, B]
      #if attackGraph(leaders[day, B]).inDegree() > #N attacchi
        #if leaders[day + 1, B] != leaders[day, B]
          #print('B has a new leader after the war at day: ' +  day)
          
          
def search_for_leaders(comm_dict, allianceGraph, AttackGraph)
  #leaders = dict()
  #for node in alliance
    #if node.centrality is
        

IndentationError: expected an indented block (<ipython-input-73-9efbbd21ac2d>, line 11)

#### Community Centrality

In [11]:
def findDiplomats(dayGraph):
    print("DIPLOMATS DETECTION")
    edgeBet = dayGraph.edge_betweenness(weights = dayGraph.es['Weight'])
    # print(edgeBet)
    orderEdgeBet = np.argsort(edgeBet)[::-1]
    print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
      format("From", "To", "From Community", "To Community", "Edge Betweennees", "Weight"))
    for e in orderEdgeBet:
        if dayGraph.vs[dayGraph.es[e].tuple[0]]['Community'] != dayGraph.vs[dayGraph.es[e].tuple[1]]['Community']:
            print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
                format(dayGraph.vs[dayGraph.es[e].tuple[0]]['id'], dayGraph.vs[dayGraph.es[e].tuple[1]]['id'], 
                         dayGraph.vs[dayGraph.es[e].tuple[0]]['Community'], dayGraph.vs[dayGraph.es[e].tuple[1]]['Community'], 
                         round(edgeBet[e], 5), round(dayGraph.es[e]['Weight']), 0))

### Visualize Network
TODO

In [75]:
def VisualizeNetwork(allianceNet):
    print("USING Plotly Python Open Source Graphing Library")

## Main
From cell below we let's start all our analysis

In [22]:
#      TRY, IF YOU WANT!      #
 #############################
#userNumberTrend(allNetwork)  #
#edgeNumberTrend(allNetwork)  #
tricksterAnalysis(allNetwork)#
 #############################
communityAffiliation = dict()

'''for i in range(0, 1):
    # print(allNetwork[i][4].summary())
    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][1], i)
    # addCommunitiesLabel(allNetwork[i][3], i)
    print(allNetwork[i][1].summary())
print("")

# print(communityAffiliation)
    

findDiplomats(allNetwork[0][1])'''

# for vertex in allNetwork[i][5].vs:
#    print(vertex)
# print(allNetwork[1][5].vs[2])
# print(len(allNetwork[1][5].es))'''



Start trickester Analysis
Watching for dishonest user in the period...



Compact recovered information...
This are the main suspects!
GULTY INDEX:  1.2752
SUSPECT:  7855
EDGES IN:  {'Attack': 3.0, 'Trade': 0, 'Message': 0}
EDGES OUT:  {'Attack': 0, 'Trade': 22.0, 'Message': 0}
PLAYER SERVED:  {'3778': 22.0}
ACTIVITY DAYS:  ['1', '3', '6', '7', '8', '10', '11', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']
DAYS OF ATTACKS SUFFERED:  [6, 7]
DAYS OF TRADE DONE:  [1, 3, 8, 10, 11, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
PLAYER TRADE WITH SUSPECT:  {}
 
GULTY INDEX:  1.2792
SUSPECT:  2789
EDGES IN:  {'Attack': 13.0, 'Trade': 0, 'Message': 0}
EDGES OUT:  {'Attack': 0, 'Trade': 114.0, 'Message': 0}
PLAYER SERVED:  {'1451': 114.0}
ACTIVITY DAYS:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '28']
DAYS OF ATTACKS SUFFERED:  [8, 10, 12, 18, 19, 21, 22, 24, 25, 26, 28]
DAYS OF TRADE DONE:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,

GULTY INDEX:  1.2403
SUSPECT:  12618
EDGES IN:  {'Attack': 2.0, 'Trade': 0, 'Message': 0}
EDGES OUT:  {'Attack': 0, 'Trade': 6.0, 'Message': 0}
PLAYER SERVED:  {'12435': 6.0}
ACTIVITY DAYS:  ['27', '29', '30']
DAYS OF ATTACKS SUFFERED:  [27]
DAYS OF TRADE DONE:  [29, 30]
PLAYER TRADE WITH SUSPECT:  {}
 
20


'for i in range(0, 1):\n    # print(allNetwork[i][4].summary())\n    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][1], i)\n    # addCommunitiesLabel(allNetwork[i][3], i)\n    print(allNetwork[i][1].summary())\nprint("")\n\n# print(communityAffiliation)\n    \n\nfindDiplomats(allNetwork[0][1])'

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete